In [ ]:
# import matplotlib.pyplot as plt
import numpy as np
import pprint

import sys
import os

sys.path.append(os.path.abspath('..'))

from SafeRLBench import config
from SafeRLBench import Bench, BenchConfig, BestPerformance
from SafeRLBench.algo import PolicyGradient
from SafeRLBench.envs import LinearCar, GeneralMountainCar
from SafeRLBench import envs, tools, algo
    
np.set_printoptions(precision=5)

In [ ]:
config.logger_set_level()
config.logger_add_stream_handler()
# config.logger_add_file_handler('BenchTestLog.log')
config.monitor_set_verbosity(2)
config.jobs_set(1)

In [ ]:
# configure a policy
def policy_par(par):
    def policy(x):
        return np.exp(par[0] - par[1:].dot(x)) - 1.5
    return policy
policy = tools.Policy(policy_par, (3))

def policy_grad(x, a, par):
    return np.array([1., -x[0], -x[1]]) * policy_par(par)(x)

policy_rif = tools.Policy(policy_par,(3), gradient=policy_grad, sigma=0.2)

# configure environments
algs = [
    (PolicyGradient, [{'policy': policy, 'max_it': 1000, 'eps': 0.00001, 'estimator': 'central_fd'},
                      {'policy': policy, 'max_it': 2000, 'eps': 0.00001, 'estimator': 'central_fd'}])
]

env = [[
    (LinearCar, {'horizon': 200}),
    (LinearCar, {'horizon': 100})
]]


test_config = BenchConfig(algs, env)

In [ ]:
benchmark = Bench(test_config, [BestPerformance()])

In [ ]:
benchmark()

In [ ]:
benchmark.measures[0].out

In [ ]:
benchmark.runs